# MODEL BUILDING

In [20]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='ZIMl8dwkeQKnmzXuunZiAflAyM_200EIno1-EuJqSetN',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'emergingmethodsforearlydetectiono-donotdelete-pr-jakd14v7zyhxir'
object_key = 'archive.zip'

streaming_body_1 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [21]:
from io import BytesIO
import zipfile
unzip=zipfile.ZipFile(BytesIO(streaming_body_1.read()),'r')
file_paths=unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [22]:
ls

Dataset/  forest1.h5


In [25]:
import keras
from keras.preprocessing.image import ImageDataGenerator

In [68]:
train_datagen=ImageDataGenerator(rescale=1./255,
                                 shear_range=0.2,
                                 rotation_range=180,
                                 zoom_range=0.2,
                                 horizontal_flip=True)
test_datagen=ImageDataGenerator(rescale=1./255)

In [40]:
import tensorflow as tf

In [41]:
x_test = test_datagen.flow_from_directory('Dataset/Dataset/test_set',
                                          target_size=(128,128),
                                          batch_size=32,
                                          class_mode='binary')

Found 121 images belonging to 2 classes.


In [ ]:
x_train = train_datagen.flow_from_directory('Dataset/Dataset/train_set',
                                          target_size=(128,128),
                                          batch_size=32,
                                          class_mode='binary')

In [ ]:
#import model building libraries

#to define linear initialisation import sequential
from keras.models import Sequential
#to add layer import Dense
from keras.layers import Dense
#to create convolution kernel import convolution2D
from keras.layers import Convolution2D
#import Maxpooling layer
from keras.layers import MaxPool2D
#import flatten layer
from keras.layers import Flatten
from keras.layers import Dropout
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#initialize the model
model=Sequential()

In [60]:
#from keras.layers.pooling.max_pooling2d import MaxPooling2D
from tensorflow.keras.layers import MaxPooling2D, Dropout, Flatten, Dense
#add convolution layer
model.add(Convolution2D(32,(3,3),input_shape=(128,128,3),activation='relu'))
#add maxpooling layer
model.add(MaxPooling2D(pool_size=(2,2)))
#add flatten layer
model.add(Flatten())

In [61]:
#add hidden layer
model.add(Dense(150,activation='relu'))
#add output layer
model.add(Dense(1,activation='sigmoid'))

In [62]:
#configure the learning process
model.compile(loss = 'binary_crossentropy',
              optimizer = "adam",
              metrics = ["accuracy"])

In [64]:
#training the model
model.fit_generator(x_train,steps_per_epoch=14,
                    epochs=10,validation_data=x_test,
                    validation_steps=4)

Epoch 1/10
14/14 [==============================] - 21s 2s/step - loss: 0.1920 - accuracy: 0.9220 - val_loss: 0.1805 - val_accuracy: 0.9421
Epoch 2/10
14/14 [==============================] - 21s 2s/step - loss: 0.1860 - accuracy: 0.9083 - val_loss: 0.0491 - val_accuracy: 0.9752
Epoch 3/10
14/14 [==============================] - 21s 1s/step - loss: 0.2084 - accuracy: 0.9174 - val_loss: 0.0806 - val_accuracy: 0.9669
Epoch 4/10
14/14 [==============================] - 20s 1s/step - loss: 0.1951 - accuracy: 0.9174 - val_loss: 0.0512 - val_accuracy: 0.9752
Epoch 5/10
14/14 [==============================] - 20s 1s/step - loss: 0.1570 - accuracy: 0.9358 - val_loss: 0.0512 - val_accuracy: 0.9752
Epoch 6/10
14/14 [==============================] - 21s 1s/step - loss: 0.1633 - accuracy: 0.9174 - val_loss: 0.0813 - val_accuracy: 0.9752
Epoch 7/10
14/14 [==============================] - 20s 1s/step - loss: 0.1585 - accuracy: 0.9243 - val_loss: 0.0613 - val_accuracy: 0.9752
Epoch 8/10
14/14 [==

In [65]:
#save the model
model.save("forest1.h5")

In [69]:
#import load_model from keras.model
from keras.models import load_model
#import image class from keras
from tensorflow.keras.preprocessing import image
#import numpy
import numpy as np
#import cv2
import cv2

In [71]:
model = load_model("forest1.h5")

In [73]:
img=image.load_img('Dataset/Dataset/test_set/with fire/The_Rim_Fire_in_the_Stanislaus_National_Forest_near_in_California_began_on_Aug._17__2013_0004.jpg')
x=image.img_to_array(img)
res = cv2.resize(x, dsize=(128,128),interpolation=cv2.INTER_CUBIC)
x=np.expand_dims(res,axis=0)

In [74]:
pred = model.predict(x)



In [75]:
pred

array([[1.]], dtype=float32)